In [1]:
import altair as alt
from src.spinorama.load import parse_all_speakers
from src.spinorama.contour import compute_contour, compute_isoline
from src.spinorama.graph import display_freq, display_contour, display_template1, display_vertical

df = parse_all_speakers()

Loading speaker: 
  Emotiva Airmotive 6s, 
  Harbeth Monitor, 
  JBL 104, 
  KEF LS50, 
  Kali IN-8, 
  Micca RB42, 
  Neumann KH 80, 
  Pioneer SP-BS22-LR, 
  Realistic MC-1000, 
  Revel C52, 
  Selah Audio RC3R, 
  Yamaha HS5, 


In [2]:
for k in df.keys():
  print(k)

Emotiva Airmotive 6s
Harbeth Monitor
JBL 104
KEF LS50
Kali IN-8
Micca RB42
Neumann KH 80
Pioneer SP-BS22-LR
Realistic MC-1000
Revel C52
Selah Audio RC3R
Yamaha HS5


In [3]:
neumann = df['Neumann KH 80']
for k in neumann.keys():
    print(k)

CEA2034
CEA2034_unmelted
Early Reflections
Early Reflections_unmelted
Estimated In-Room Response
Estimated In-Room Response_unmelted
Horizontal Reflections
Horizontal Reflections_unmelted
Vertical Reflections
Vertical Reflections_unmelted
SPL Horizontal
SPL Horizontal_unmelted
SPL Vertical
SPL Vertical_unmelted


In [4]:
spinorama_u = neumann['CEA2034_unmelted']
for m in spinorama_u.keys():
    print(m)
spinorama = neumann['CEA2034']
for m in spinorama.keys():
    print(m)

Freq
On Axis
Listening Window
Early Reflections
Sound Power
Sound Power DI
Early Reflections DI
DI offset
Freq
Measurements
dB


In [5]:
onaxis = spinorama.loc[spinorama['Measurements']=='On Axis']
print(onaxis)

           Freq Measurements        dB
200     20.5078      On Axis   55.7961
201     21.2402      On Axis   51.9241
202     21.9727      On Axis   53.8309
203     22.7051      On Axis   55.0830
204     23.4375      On Axis   41.6382
..          ...          ...       ...
395  17753.2000      On Axis  107.9410
396  18379.4000      On Axis  106.9740
397  19027.6000      On Axis  104.8340
398  19698.5000      On Axis  104.4100
399  19999.5000      On Axis  104.0890

[200 rows x 3 columns]


In [7]:
from src.spinorama.analysis import estimates

e0, e3, e6, delta = estimates(onaxis)

print('Estimated -3dB point at %1.fHz'% e3)
print('Estimated -6dB point at %1.fHz'% e6)
print('Estimated +/-%1.1fdB from 80hz to 20kz'%delta)


Estimated -3dB point at 58Hz
Estimated -6dB point at 55Hz
Estimated +/-3.1dB from 80hz to 20kz


In [222]:
import math
import pandas as pd
import numpy as np

spl = neumann['SPL Horizontal']

radius=1
anglelist = [a for a in range(-180,180,10)]
gridC = [(radius*math.cos(p*math.pi/180), 
          radius*math.sin(p*math.pi/180)) for p in anglelist]
gridX = [(0,g[0]) for g in gridC] 
gridX = [s for s2 in gridX for s in s2]
gridY = [(0,g[1]) for g in gridC] 
gridY = [s for s2 in gridY for s in s2]

#for g in zip(gridX, gridY):
#    print('line from %d, %d' % g)

splu = neumann['SPL Horizontal_unmelted']
# 100hz 47
#  1khz 113
# 10khz 180
def hzname(i):
    if i == 47:
        return '100 Hz'
    elif i == 113:
        return '1 kHz'
    elif i == 180:
        return '10 kHz'
    else:
        return 'error'

dbX = []
dbY = []
hz  = []
for ihz in [47, 113, 180]:
    gridZ = splu.loc[ihz][1:]
    angles = [0]
    values = {}
    dbs = []
    for a, z in zip(gridZ.index,gridZ):
        angle = 0
        if a != 'On-Axis':
            angle = int(a[:-1])
            angles.append(angle)
        dbs.append(z)

    anglemin = np.array(angles).min()/10
    # [0   1    2   3    4   5    6   7    8   9   10  11   12  13   ]
    # [0, 10, -10, 20, -20, 30, -30, 40, -40, 50, -50, 60, -60, 70, -70, 80, -80, 90, -90, 100, -100, 110, -110, 120, -120, 130, -130, 140, -140, 150, -150, 160, -160, 170, -170, 180]
    # perm[0] = orig[-170] = 35
    # perm[1] = orig[-150] = 33
    # perm[2] =            = 31
    # perm[17] = 0
    # perm[18] = 1
    # perm[19] = 3 
    def compute_perm(i):
        if i<17:
            return 34-2*i
        elif i == 17:
            return 0
        else:
            return 1+2*(i-18)
    perm = [compute_perm(i) for i in range(0, len(angles))]
    dbsmax = 1 # np.array(dbs).max()
    sorted_dbs = [dbs[perm[i]] for i in range(0,len(perm))]
    sorted_angles = [angles[perm[i]] for i in range(0,len(perm))]
    # print(angles)
    # print(perm)
    # print(sorted_angles)
        
    # map in 2d
    dbsX = [db*math.cos(a*math.pi/180) for a,db in zip(sorted_angles, sorted_dbs)]
    dbsY = [db*math.sin(a*math.pi/180) for a,db in zip(sorted_angles, sorted_dbs)]

    dbsX.append(dbsX[0])
    dbsY.append(dbsY[0])
    # add to global variable
    dbX.append(dbsX)
    dbY.append(dbsY)
    hz.append([ihz for i in range(0,len(dbsX))])

dbmax = max(np.array(dbX).max(), np.array(dbY).max())
dbX = [v2/dbmax for v1 in dbX for v2 in v1]
dbY = [v2/dbmax for v1 in dbY for v2 in v1]
hz  = [hzname(i2) for i1 in hz for i2 in i1]

grid_df = pd.DataFrame({'x': gridX, 'y': gridY })
grid = alt.Chart(grid_df).mark_line(
).encode(
   alt.Latitude('x:Q'), 
   alt.Longitude('y:Q')
).project(
   type='azimuthalEquidistant',
   rotate=[0, 0, 90]   
)

dbs_df = pd.DataFrame({'x': dbX, 'y': dbY, 'hz': hz })
dbs = alt.Chart(dbs_df).mark_line(
).encode(
   alt.Latitude('x:Q'), 
   alt.Longitude('y:Q'),
   alt.Color('hz:N')
).project(
   type='azimuthalEquidistant',
   rotate=[0, 0, 90]   
)

grid+dbs

[0, 10, -10, 20, -20, 30, -30, 40, -40, 50, -50, 60, -60, 70, -70, 80, -80, 90, -90, 100, -100, 110, -110, 120, -120, 130, -130, 140, -140, 150, -150, 160, -160, 170, -170, 180]
[34, 32, 30, 28, 26, 24, 22, 20, 18, 16, 14, 12, 10, 8, 6, 4, 2, 0, 1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 35]
[-170, -160, -150, -140, -130, -120, -110, -100, -90, -80, -70, -60, -50, -40, -30, -20, -10, 0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180]
[0, 10, -10, 20, -20, 30, -30, 40, -40, 50, -50, 60, -60, 70, -70, 80, -80, 90, -90, 100, -100, 110, -110, 120, -120, 130, -130, 140, -140, 150, -150, 160, -160, 170, -170, 180]
[34, 32, 30, 28, 26, 24, 22, 20, 18, 16, 14, 12, 10, 8, 6, 4, 2, 0, 1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 35]
[-170, -160, -150, -140, -130, -120, -110, -100, -90, -80, -70, -60, -50, -40, -30, -20, -10, 0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180]
[0, 10, -10, 20, -

alt.LayerChart(...)